# Running a UDF on aggregated time series

https://github.com/Open-EO/openeo-geopyspark-driver/blob/master/docs/vectorcube-run_udf.md

In [2]:
import openeo

In [3]:
eoconn = openeo.connect('https://openeo.vito.be').authenticate_oidc()

Authenticated using refresh token.


In [4]:
# Specific parameters
croptype = 'Zomergerst'

# Bounding Box
west = 5.17
east = 5.3
south = 51.1
north = 52.246

In [10]:
import urllib
import requests

url = f"https://geo.api.vlaanderen.be/Landbgebrperc/wfs?service=WFS&request=getfeature&cql_filter=LBLHFDTLT='{croptype}'&outputformat=json&typename=Lbgebrperc&SRSName=urn:x-ogc:def:crs:EPSG:4326"
req = requests.get(url)

wfs_request_url = req.url
data = req.json()
data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': 'Lbgebrperc.1585',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[5.24743582, 51.10311003],
      [5.24843201, 51.10333602],
      [5.24844207, 51.10333831],
      [5.24869375, 51.10339541],
      [5.24870474, 51.1033979],
      [5.24930975, 51.10353509],
      [5.24930314, 51.10362455],
      [5.24932257, 51.10371259],
      [5.24932876, 51.10373376],
      [5.24933028, 51.10373869],
      [5.2493388, 51.10383499],
      [5.24932919, 51.1038855],
      [5.2493219, 51.10392313],
      [5.24931965, 51.10393539],
      [5.24931889, 51.10393788],
      [5.24931217, 51.10395997],
      [5.24784072, 51.10361716],
      [5.24730104, 51.10349143],
      [5.24724946, 51.10347942],
      [5.24728281, 51.10340597],
      [5.24741921, 51.10310626],
      [5.24743582, 51.10311003]]]},
   'geometry_name': 'SHAPE',
   'properties': {'UIDN': 1882791,
    'OIDN': 1021017,
    'ALVID': 1319447853,
    'HFDTLT': '322',
  

In [37]:
data["features"] = data["features"][0:10]

In [12]:

sentinel2 = eoconn.load_collection(
    "TERRASCOPE_S2_TOC_V2",
    temporal_extent = ["2022-06-04", "2022-08-04"],
    bands = ["B02", "B03", "B04","SCL"],
    max_cloud_cover=75
)

sentinel2 = sentinel2.process(
            "mask_scl_dilation",
            data=sentinel2,
            scl_band_name="SCL",
            kernel1_size=17, kernel2_size=77,
            mask1_values=[2, 4, 5, 6, 7],
            mask2_values=[3, 8, 9, 10, 11],
            erosion_kernel_size=3)

sentinel2 = sentinel2.aggregate_temporal_period("dekad",reducer="median")\
    .apply_dimension(dimension="t", process="array_interpolate_linear")

### Generated timeseries 'vector cube' per parcel

In [42]:
parcel_timeseries = sentinel2.aggregate_spatial(data,reducer="median")

### Run custom Python code (UDF) on the Vector Cube

In [48]:
import pathlib
path = pathlib.Path("classify_parcel_udf.py")
code = path.read_text(encoding="utf-8")

In [49]:
result = parcel_timeseries.process("run_udf", data=parcel_timeseries, udf=code, runtime='Python', context=dict())

In [51]:
crop_calendars = result.execute_batch("out.json")

0:00:00 Job 'j-76128f548287447fb21de1ddeb3e189c': send 'start'
0:00:24 Job 'j-76128f548287447fb21de1ddeb3e189c': queued (progress N/A)
0:00:30 Job 'j-76128f548287447fb21de1ddeb3e189c': queued (progress N/A)
0:00:36 Job 'j-76128f548287447fb21de1ddeb3e189c': queued (progress N/A)
0:00:44 Job 'j-76128f548287447fb21de1ddeb3e189c': queued (progress N/A)
0:00:54 Job 'j-76128f548287447fb21de1ddeb3e189c': queued (progress N/A)
0:01:07 Job 'j-76128f548287447fb21de1ddeb3e189c': queued (progress N/A)
0:01:22 Job 'j-76128f548287447fb21de1ddeb3e189c': queued (progress N/A)
0:01:42 Job 'j-76128f548287447fb21de1ddeb3e189c': queued (progress N/A)
0:02:06 Job 'j-76128f548287447fb21de1ddeb3e189c': running (progress N/A)
0:02:36 Job 'j-76128f548287447fb21de1ddeb3e189c': running (progress N/A)
0:03:31 Job 'j-76128f548287447fb21de1ddeb3e189c': running (progress N/A)
0:04:18 Job 'j-76128f548287447fb21de1ddeb3e189c': running (progress N/A)
0:05:17 Job 'j-76128f548287447fb21de1ddeb3e189c': running (progress N